In [2]:
# Make all plots inline.
%matplotlib inline 

# Import dependencies
import folium
from folium.plugins import HeatMap
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
from IPython.core.display import display
import folium.colormap as cm

# Loading datasets

In [3]:
# Load brewery data.
pop_data = pd.read_csv('./wrk_tables/zip_features.csv')
review_data = pd.read_csv('./src_tables/review.csv')

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
beer_data = pd.read_csv('./wrk_tables/beer_eda_all.csv')

In [5]:
# Remove first column
beer_data = beer_data.iloc[:,1:len(beer_data.iloc[1])+1]

In [6]:
beer_data['id'] = beer_data['id'].str.replace(',', '')

In [7]:
pop_data.head()

,postal_code_id,year,total,total_m,total_f,under_18_m,x18_to_29_m,x30_to_49_m,x50_to_64_m,x65_plus_m,...,median_household_income,num_white_households,num_family_households,avg_household_size,num_us_citizens,num_english_speakers,num_with_american_ancestry,num_hs_grads,num_bach_degree,num_post_grad
0,147.0,2011,18533.0,8971,9562.0,2476,1616,2248,1551,1080,...,13318.0,482967.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,147.0,2012,18544.0,9040,9504.0,2396,1633,2228,1636,1147,...,13495.0,511004.0,NaN,NaN,NaN,NaN,NaN,4053.0,1595.0,272.0
2,147.0,2013,18450.0,8983,9467.0,2319,1576,2260,1651,1177,...,12041.0,549100.0,NaN,NaN,NaN,NaN,NaN,3957.0,1630.0,271.0
3,147.0,2014,18088.0,8824,9264.0,2215,1518,2204,1681,1206,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3693.0,1617.0,361.0
4,147.0,2015,17982.0,8788,9194.0,2142,1531,2138,1722,1255,...,10833.0,561048.0,NaN,NaN,NaN,NaN,NaN,3788.0,1720.0,372.0


In [8]:
review_data.head()

,id,beer_id,date,rating,text,user_name,location_id,country
0,"47,879",452,2010-03-26,3.9,Pours brown/black with a thin brown/beige head...,Gustav1996,"11,242",USA
1,"47,880",719,2015-06-21,4.2,On tap. Pours clear red with tan head. Aroma i...,Vestlake,"11,242",USA
2,"47,881",288,2008-06-03,4.2,Bottle. A dark orange beer with no head. The a...,CaptBier,"11,242",USA
3,"47,882","4,357",2013-12-22,3.6,Bottle. Poured amber-brown with a thinnish tan...,coolpop,"11,242",USA
4,"47,883","1,804",2010-02-25,3.2,"UPDATED: MAR 14, 2010 33cl bottle. Poured a oi...",Gustav1996,"11,242",USA


In [9]:
yelp_data.head()

,uid,name,price,rating,review_count,city,state,zip_code,latitude,longitude,name_norm,id,brewery_id
0,airways-brewing-kent,Airways Brewing,$,4.5,53,Kent,WA,98031,47.411718,-122.224823,Airways Brewing Company,"4,869",104
1,airways-brewing-the-bistro-in-downtown-kent-kent,Airways Brewing: The Bistro in Downtown Kent,$$,4.0,182,Kent,WA,98032,47.382172,-122.235910,Airways Brewing Company,"4,862",104
2,akasha-brewing-co-louisville-2,Akasha Brewing Co,$,4.5,43,Louisville,KY,40206,38.253129,-85.735413,Akasha Brewing Company,"6,664",107
3,alameda-island-brewing-company-alameda,Alameda Island Brewing Company,$,4.5,190,Alameda,CA,94501,37.768440,-122.238861,Alameda Island Brewing Company,"1,396",115
4,alamo-beer-san-antonio,Alamo Beer,$$,4.0,79,San Antonio,TX,78202,29.429416,-98.477224,Alamo Beer Company,919,116


In [10]:
# Replacing missing IBU values with mean value for corresponding beer style
for i in range(0, len(beer_data['id'])):
    if np.isnan(beer_data['ibu'].iloc[i]):
        beer_data['ibu'].iloc[i] = np.nanmean(beer_data['ibu'].loc[beer_data['beer_style_norm'] == beer_data['beer_style_norm'].iloc[i]])

//anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
//anaconda/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:703: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


In [11]:
# None of the Sakes have an IBU, so replace with 0
beer_data['ibu'] = beer_data['ibu'].fillna(0)

In [12]:
# Replacing missing calorie values with mean value for corresponding beer style
for i in range(0, len(beer_data['id'])):
    if np.isnan(beer_data['calories'].iloc[i]):
        beer_data['calories'].iloc[i] = np.nanmean(beer_data['calories'].loc[beer_data['beer_style_norm'] == beer_data['beer_style_norm'].iloc[i]])

In [13]:
# Replacing missing ABV values with mean value for corresponding beer style
for i in range(0, len(beer_data['id'])):
    if np.isnan(beer_data['abv'].iloc[i]):
        beer_data['abv'].iloc[i] = np.nanmean(beer_data['abv'].loc[beer_data['beer_style_norm'] == beer_data['beer_style_norm'].iloc[i]])

In [14]:
beer_data.to_csv('./src_dtables/beer_data_all_v2.csv')

# City Geo Data 

In [15]:
from urllib2 import Request, urlopen
import json
from pandas.io.json import json_normalize

In [16]:
with open('./src_tables/cities.json') as data_file:    
    city_coord = json.load(data_file)

city_coord = json_normalize(city_coord)
city_coord.to_csv('./src_tables/city_coord.csv')

In [17]:
city_pop_16 = pd.read_csv('./src_tables/city_pop_2016.csv')

In [18]:
city_merge = pd.merge(city_coord, city_pop_16, left_on = ('city','state'), right_on = ('City','State'), left_index = False)

In [19]:
city_merge.head()

,city,growth_from_2000_to_2013,latitude,longitude,population,rank,state,Rank,City,State,Census_2010,Base_2010,July_2010,July_2011,July_2012,July_2013,July_2014,July_2015,July_2016
0,New York,4.8%,40.712784,-74.005941,8405837,1,New York,1,New York,New York,8175133,8174962,8192026,8284098,8361179,8422460,8471990,8516502,8537673
1,Los Angeles,4.8%,34.052234,-118.243685,3884307,2,California,2,Los Angeles,California,3792621,3792584,3796292,3825393,3858137,3890436,3920173,3949149,3976322
2,Chicago,-6.1%,41.878114,-87.629798,2718782,3,Illinois,3,Chicago,Illinois,2695598,2695620,2697736,2705404,2714120,2718887,2718530,2713596,2704958
3,Houston,11.0%,29.760427,-95.369803,2195914,4,Texas,4,Houston,Texas,2099451,2100277,2105625,2132157,2166458,2204406,2243999,2284816,2303482
4,Philadelphia,2.6%,39.952584,-75.165222,1553165,5,Pennsylvania,6,Philadelphia,Pennsylvania,1526006,1526006,1528427,1539022,1550379,1555868,1560609,1564964,1567872


In [20]:
# removing unwanted columns
city_data = city_merge[['Rank','city','state','latitude','longitude','July_2016']]

In [21]:
# Adding column for number of breweries
city_data['breweries_5'] = [0] * len(city_data)
city_data['breweries_10'] = [0] * len(city_data)
city_data['breweries_25'] = [0] * len(city_data)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas

In [24]:
city_data = city_data.sort_values(by = 'Rank', ascending=False)

In [163]:
# Adding column for number of breweries
city_data['ratings_5'] = [0] * len(city_data)
city_data['ratings_10'] = [0] * len(city_data)
city_data['ratings_25'] = [0] * len(city_data)

city_data['review_count_5'] = [0] * len(city_data)
city_data['review_count_10'] = [0] * len(city_data)
city_data['review_count_25'] = [0] * len(city_data)

city_data['price_avg_5'] = [0] * len(city_data)
city_data['price_avg_10'] = [0] * len(city_data)
city_data['price_avg_25'] = [0] * len(city_data)

# Working with Yelp Data

In [ ]:
yelp_data = pd.read_csv('./wrk_tables/yelp_brewery.csv')

In [ ]:
# Function calculating the Bayesian weighted average yelp rating
# S = wR + (1-w)C
# w = /(v+m), where v = num ratings for each individual beer, and 
# m = avg num ratings for city
# R = overall_rating for each individual beer
# C = average overall_rating for the city
def wtd_yelp_rating():
    rating_list = [0] * len(yelp_data)
    for i in range(0,len(yelp_data)):
        v = yelp_data['review_count'].iloc[i]
        m = np.mean(yelp_data['review_count'].loc[(yelp_data['city'] == yelp_data['city'].iloc[i]) & (yelp_data['state'] == yelp_data['state'].iloc[i])])
        w = v/(v+m)
        R = yelp_data['rating'].iloc[i]
        C = np.mean(yelp_data['rating'].loc[(yelp_data['city'] == yelp_data['city'].iloc[i]) & (yelp_data['state'] == yelp_data['state'].iloc[i])])
        rating_list[i] = w*R + (1-w)*C
    return rating_list

In [ ]:
yelp_data['wtd_rating'] = wtd_yelp_rating()

In [ ]:
# Checking for duplicates, and there are 451
pd.concat(g for _, g in yelp_data.groupby(["name_norm","city","state"]) if len(g) > 1)

In [ ]:
# Converting price '$' to interval data
yelp_data['price'] = yelp_data['price'].map({'$$$$': 4, '$$$': 3, '$$': 2, '$': 1})

In [ ]:
# Replacing missing Price values with mean value for city/state
for i in range(0, len(yelp_data)):
    if np.isnan(yelp_data['price'].iloc[i]):
        yelp_data['price'].iloc[i] = np.nanmean(yelp_data['price'].loc[(yelp_data['city'] == yelp_data['city'].iloc[i])
                                                                      & (yelp_data['state'] == yelp_data['state'].iloc[i])])

In [ ]:
# 197 breweries in cities without price information
np.sum(np.isnan(yelp_data['price']))

In [ ]:
# Since they are likely in small cities, replace remaining NaNs with 1
yelp_data['price'] = yelp_data['price'].fillna(1)

In [165]:
# columns for count of beers by brewery for each style
yelp_data['other'] = [0]*len(yelp_data)
yelp_data['sour'] = [0]*len(yelp_data)
yelp_data['wheat'] = [0]*len(yelp_data)
yelp_data['saison'] = [0]*len(yelp_data)
yelp_data['pale'] = [0]*len(yelp_data)
yelp_data['ipa'] = [0]*len(yelp_data)
yelp_data['strong'] = [0]*len(yelp_data)
yelp_data['brown'] = [0]*len(yelp_data)
yelp_data['porter'] = [0]*len(yelp_data)
yelp_data['stout'] = [0]*len(yelp_data)
yelp_data['bock'] = [0]*len(yelp_data)
yelp_data['lager'] = [0]*len(yelp_data)
yelp_data['sake'] = [0]*len(yelp_data)

# columns for count of beers by brewery for each ABV strength
yelp_data

,uid,name,price,rating,review_count,city,state,zip_code,latitude,longitude,name_norm,id,brewery_id,wtd_rating
0,airways-brewing-kent,Airways Brewing,1.0,4.5,53,Kent,WA,98031,47.411718,-122.224823,Airways Brewing Company,"4,869",104,4.194444


In [233]:
yelp_data[400:450]

,uid,name,price,rating,review_count,city,state,zip_code,latitude,longitude,name_norm,id,brewery_id,wtd_rating
400,black-bridge-brewery-kingman,Black Bridge Brewery,1.0,4.5,79,Kingman,AZ,86401,35.189636,-114.051803,Black Bridge Brewery (AZ),"7,706","1,411",4.634841
401,black-cap-brewing-company-red-lion,Black Cap Brewing Company,1.0,5.0,8,Red Lion,PA,17356,39.902489,-76.609001,Black Cap Brewing Company,"1,261","1,413",5.000000
402,black-circle-brewing-company-indianapolis-2,Black Circle Brewing Company,1.0,4.5,32,Indianapolis,IN,46205,39.839807,-86.123866,Black Circle Brewing Company,"3,277","1,416",4.193323
403,black-cloister-brewing-company-toledo,Black Cloister Brewing Company,2.0,4.5,47,Toledo,OH,43604,41.649780,-83.539804,Black Cloister Brewing Company,"5,934","1,417",4.228599
404,black-diamond-brewing-co-concord,Black Diamond Brewing Co,1.0,4.5,44,Concord,CA,94520,38.013880,-122.030830,Black Diamond Brewing Company,"1,427","1,420",4.183252
405,black-eagle-brewery-black-eagle,Black Eagle Brewery,3.0,4.5,8,Black Eagle,MT,59414,47.527832,-111.280446,Black Eagle Brewery,"6,857","1,427",4.500000
406,black-flag-brewing-company-columbia-2,Black Flag Brewing Company,1.0,4.5,55,Columbia,MD,21046,39.174430,-76.834150,Black Flag Brewing Company,"3,510","1,429",4.080853
407,black-forest-brew-haus-farmingdale,Black Forest Brew Haus,2.0,3.0,225,Farmingdale,NY,11735,40.755160,-73.414100,Black Forest Brew Haus,"3,137","1,431",3.224374
408,appalachian-brewing-company-gettysburg-2,Appalachian Brewing Company,2.0,3.0,106,Gettysburg,PA,17325,39.842617,-77.191948,Appalachian Brewing Company,32,NaN,3.328743
409,black-forest-brewery-ephrata,Black Forest Brewery,1.0,4.0,10,Ephrata,PA,17522,40.181360,-76.181330,Black Forest Brewery,676,"1,432",4.000000


In [182]:
# consider filtering for yelp breweries with at least 10 or 15 reviews
len(yelp_data.loc[yelp_data['review_count'] >= 15])

7565

In [242]:
city_rating = yelp_data.groupby(['city', 'state'])[['wtd_rating']].mean()

uid             alameda-island-brewing-company-alameda
name                    Alameda Island Brewing Company
price                                                1
rating                                             4.5
review_count                                       190
city                                           Alameda
state                                               CA
zip_code                                         94501
latitude                                       37.7684
longitude                                     -122.239
name_norm               Alameda Island Brewing Company
id                                               1,396
brewery_id                                         115
wtd_rating                                         4.5
Name: 3, dtype: object

# Calculating Distances

In [278]:
# defining distance calculation
def brewery_count():
    R = 3961 # radius of earth in miles
    for i in range(0,len(city_data)):
        for j in range(0,len(yelp_data)):
            lat1 = np.radians(city_data['latitude'].iloc[i])
            lat2 = np.radians(yelp_data['latitude'].iloc[j])
            lon1 = np.radians(city_data['longitude'].iloc[i])
            lon2 = np.radians(yelp_data['longitude'].iloc[j])
            dlon = lon2 - lon1 
            dlat = lat2 - lat1 
            a = (np.sin(dlat/2))**2 + np.cos(lat1) * np.cos(lat2) * (np.sin(dlon/2))**2 
            c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a)) 
            d = R * c
            if d <= 5:
                city_data['breweries_5'].iloc[i] += 1
                city_data['breweries_10'].iloc[i] += 1
                city_data['breweries_25'].iloc[i] += 1
                
                city_data['ratings_5'] += yelp_data['wtd_rating'].iloc[i]
                city_data['ratings_10'] += yelp_data['wtd_rating'].iloc[i]
                city_data['ratings_25'] += yelp_data['wtd_rating'].iloc[i]

                city_data['review_count_5'] += yelp_data['review_count'].iloc[i]
                city_data['review_count_10'] += yelp_data['review_count'].iloc[i]
                city_data['review_count_25'] += yelp_data['review_count'].iloc[i]

                city_data['price_5'] += yelp_data['price'].iloc[i]
                city_data['price_10'] += yelp_data['price'].iloc[i]
                city_data['price_25'] += yelp_data['price'].iloc[i]
            elif d <= 10:
                city_data['breweries_10'].iloc[i] += 1
                city_data['breweries_25'].iloc[i] += 1
                
                city_data['ratings_10'] += yelp_data['wtd_rating'].iloc[i]
                city_data['ratings_25'] += yelp_data['wtd_rating'].iloc[i]
                
                city_data['review_count_10'] += yelp_data['review_count'].iloc[i]
                city_data['review_count_25'] += yelp_data['review_count'].iloc[i]
                
                city_data['price_10'] += yelp_data['price'].iloc[i]
                city_data['price_25'] += yelp_data['price'].iloc[i]
            elif d <= 25:
                city_data['breweries_25'].iloc[i] += 1
                
                city_data['ratings_25'] += yelp_data['wtd_rating'].iloc[i]
                
                city_data['review_count_25'] += yelp_data['review_count'].iloc[i]
                    
                city_data['price_25'] += yelp_data['price'].iloc[i]
            else:
                pass
    city_data['ratings_5'] =  city_data['ratings_5']/city_data['breweries_5']
    city_data['ratings_10'] =  city_data['ratings_10']/city_data['breweries_10']
    city_data['ratings_25'] =  city_data['ratings_25']/city_data['breweries_25']

    city_data['review_count_5'] = city_data['review_count_5']/city_data['breweries_5']
    city_data['review_count_10'] = city_data['review_count_10']/city_data['breweries_10']
    city_data['review_count_25'] = city_data['review_count_25']/city_data['breweries_25']

    city_data['price_5'] = city_data['price_5']/city_data['breweries_5']
    city_data['price_10'] = city_data['price_10']/city_data['breweries_10']
    city_data['price_25'] = city_data['price_25']/city_data['breweries_25']
    return(city_data)

In [27]:
city_data = brewery_count()

In [29]:
city_data = city_data.sort_values(by = 'Rank', ascending=True)

In [31]:
city_data.head(50)

,Rank,city,state,latitude,longitude,July_2016,breweries_5,breweries_10,breweries_25
0,1,New York,New York,40.712784,-74.005941,8537673,29,49,112
1,2,Los Angeles,California,34.052234,-118.243685,3976322,16,45,144
2,3,Chicago,Illinois,41.878114,-87.629798,2704958,22,41,112
3,4,Houston,Texas,29.760427,-95.369803,2303482,25,36,67
5,5,Phoenix,Arizona,33.448377,-112.074037,1615017,12,33,80
4,6,Philadelphia,Pennsylvania,39.952584,-75.165222,1567872,16,36,98
6,7,San Antonio,Texas,29.424122,-98.493628,1492510,19,28,41
7,8,San Diego,California,32.715738,-117.161084,1406630,47,85,149
8,9,Dallas,Texas,32.776664,-96.796988,1317929,28,31,81
9,10,San Jose,California,37.338208,-121.886329,1025350,17,23,60


In [36]:
city_data['brew_per_100k_5'] = city_data['breweries_5']/city_data['July_2016']*100000
city_data['brew_per_100k_10'] = city_data['breweries_10']/city_data['July_2016']*100000
city_data['brew_per_100k_25'] = city_data['breweries_25']/city_data['July_2016']*100000

In [87]:
city_data_sort = city_data.sort_values(by = 'brew_per_100k_5', ascending=False)
city_data_sort.iloc[25:50]

,Rank,city,state,latitude,longitude,July_2016,breweries_5,breweries_10,breweries_25,brew_per_100k_5,brew_per_100k_10,brew_per_100k_25
759,755,Coeur d'Alene,Idaho,47.677683,-116.780466,50285,13,17,29,25.852640,33.807298,57.671274
649,679,Kettering,Ohio,39.689504,-84.168827,55306,14,23,34,25.313709,41.586808,61.476151
345,327,San Marcos,California,33.143372,-117.166145,95261,24,44,128,25.193941,46.188892,134.367685
389,376,Bellingham,Washington,48.749080,-122.478147,87574,22,24,40,25.121611,27.405394,45.675657
429,469,Camden,New Jersey,39.925946,-75.119620,74420,18,33,84,24.187046,44.342919,112.872884
625,585,Redmond,Washington,47.673988,-122.121512,62458,15,38,164,24.016139,60.840885,262.576451
572,561,West Des Moines,Iowa,41.577211,-93.711332,64560,15,25,31,23.234201,38.723668,48.017348
489,486,Missoula,Montana,46.878718,-113.996586,72364,16,17,18,22.110442,23.492344,24.874247
603,629,Lancaster,Pennsylvania,40.037875,-76.305514,59218,13,18,51,21.952785,30.396163,86.122463
756,746,Olympia,Washington,47.037874,-122.900695,51202,11,13,27,21.483536,25.389633,52.732315


# Storing User Beer

In [44]:
beer_data.loc[beer_data['name'] == 'Dogfish Head 90 Minute Imperial IPA'].index

Int64Index([1165], dtype='int64')

In [45]:
# User searches for beer in database (with autocomplete)
# Here use 'Dogfish Head 90 Minute Imperial IPA' as example (beer_id = 1)
USER_BEER = beer_data.iloc[1165]
USER_BEER

id                                                                1195
brewery_id                                                       4,327
beer_style_id                                                       30
brewery_type_id                                                      4
post_code                                                       24,767
loc_id                                                           17899
name                               Dogfish Head 90 Minute Imperial IPA
description          Esquire Magazine calls our 90 Minute I.I.PA., ...
abv                                                                  9
calories                                                           270
ibu                                                                 90
num_ratings                                                       4646
overall_rating                                                     100
beer_style_rating                                                   99
mean_r

In [46]:
USER_BEER_ID = USER_BEER['id']

# Calculating Style Similarity Score

In [47]:
# Calculate standard deviations of ABV, Calories, IBU, Style Score and Style Rating
abv_std = np.std(beer_data['abv'])
cal_std = np.std(beer_data['calories'])
ibu_std = np.std(beer_data['ibu'])
style_std = np.std(beer_data['style_score'])
style_rat_std = np.std(beer_data['beer_style_rating'])

In [48]:
# Determine relative weights
# I think similarity of style and ABV should be most important
# Then style rating, calories and IBUs (because many IBUs are just the mean)

style_wt = 0.25
abv_wt = 0.3
cal_wt = 0.15
ibu_wt = 0.15
style_rat_wt = 0.15

In [49]:
# Function running the algorithm for getting style similarity scores
# Runs quickly, so no need to pre-compute I think
def style_similarity(user_beer):
    sim_list = [0] * len(beer_data['id'])
    for i in range(0,len(beer_data['id'])):
        if user_beer['id'] == beer_data['id'].iloc[i]:
            sim_list[i] = np.nan
        else:
            dev = np.sqrt(style_wt*((user_beer['style_score'] - beer_data['style_score'].iloc[i])/style_std)**2 + 
                          abv_wt*((user_beer['abv'] - beer_data['abv'].iloc[i])/abv_std)**2 + 
                          cal_wt*((user_beer['calories'] - beer_data['calories'].iloc[i])/cal_std)**2 + 
                          ibu_wt*((user_beer['ibu'] - beer_data['ibu'].iloc[i])/ibu_std)**2 + 
                          style_rat_wt*((user_beer['beer_style_rating'] - beer_data['beer_style_rating'].iloc[i])/style_rat_std)**2)
            sim_score = 100*((1.25-dev)/1.25) # Aiming to have ~80% of beers above 0, depending on style
            sim_list[i] = sim_score
    return sim_list

In [50]:
# Generate Style similarity scores for the test case
SIMILARITY_SCORES = style_similarity(USER_BEER)

In [51]:
# Using this to return index of max value
import operator
index, value = max(enumerate(SIMILARITY_SCORES), key=operator.itemgetter(1))
print(index, value)

(3661, 94.442906407451559)


In [52]:
# Best similarity match
SIMILARITY_SCORES[1783]

18.993169509063446

In [53]:
beer_data.iloc[1783]

id                                                                1834
brewery_id                                                       5,142
beer_style_id                                                       53
brewery_type_id                                                      6
post_code                                                       13,093
loc_id                                                           22816
name                   Flossmoor Station Killer Kowalski Baltic Porter
description          Our deep, dark, full-bodied lager. A slightly ...
abv                                                                  8
calories                                                           240
ibu                                                            48.8548
num_ratings                                                        220
overall_rating                                                      98
beer_style_rating                                                   94
mean_r

# Rating Component

In [54]:
# Function calculating the Bayesian weighted average overall rating
# S = wR + (1-w)C
# w = /(v+m), where v = num ratings for each individual beer, and 
# m = avg num ratings for beer style
# R = overall_rating for each individual beer
# C = average overall_rating for the beer style
def weighted_rating(rtg_measure):
    rating_list = [0] * len(beer_data['id'])
    for i in range(0,len(beer_data['id'])):
        v = beer_data['num_ratings'].iloc[i]
        m = np.mean(beer_data['num_ratings'].loc[beer_data['beer_style_norm'] == beer_data['beer_style_norm'].iloc[i]])
        w = v/(v+m)
        R = beer_data[rtg_measure].iloc[i]
        C = np.mean(beer_data[rtg_measure].loc[beer_data['beer_style_norm'] == beer_data['beer_style_norm'].iloc[i]])
        rating_list[i] = w*R + (1-w)*C
    return rating_list

In [55]:
# Compile ratings based on overall_rating
RATING_SCORES = weighted_rating('overall_rating')

In [56]:
RATING_SCORES[1783]

98.337009007942669

In [57]:
RATING_SCORES[1165]

99.909981663540222

# Calculate Preference Match Component

In [60]:
# Rohan's preferencing matching table loaded here
# containing top 50 matches for each beer_id
# all others would receive a score of 0

PREF_TABLE = pd.read_csv('./wrk_tables/beer_similarities_complete.csv')


In [61]:
# Function to calculate similarity scores for user input beer
def review_match(user_beer):
    pref_list = [0] * len(beer_data['id'])
    pref_list[0] = 0
    pref_scores = PREF_TABLE.loc[PREF_TABLE['beer_id'] == int(user_beer['id'])]
    for i in range(1,len(beer_data)):
        if beer_data['id'].iloc[i] == user_beer['id']:
            pref_list[i] = np.nan
        elif int(beer_data['id'].iloc[i]) not in pref_scores['similar_beer_id'].values:
            pref_list[i] = 0
        else:
            x = pref_scores['similarity_score'].loc[pref_scores['similar_beer_id'] == int(beer_data['id'].iloc[i])].values
            pref_list[i] = x[0]
    pref_list = [i * 100 for i in pref_list]
    return pref_list

In [62]:
PREF_SCORES = review_match(USER_BEER)

In [76]:
len([i for i in PREF_SCORES if i == 0])

854

In [990]:
# Find the most comparable beer based on reviews
index, value = max(enumerate(PREF_SCORES), key=operator.itemgetter(1))
print(index, value)

(1178, 97.152757600000001)


In [991]:
# Check if this makes sense
beer_data.iloc[1178]
# Based on a quick glance over RateBeer descriptions, I think this is a pretty good match
# Despite completely different styles (Imperial IPA vs. Doppelbock), there are similarities

id                                                                1210
brewery_id                                                         667
beer_style_id                                                       66
brewery_type_id                                                      4
post_code                                                       34,355
loc_id                                                            6960
name                 Ballast Point Navigator Doppelbock - Brandy Ba...
description          Navigator Doppelbock is our interpretation of ...
abv                                                                 10
calories                                                           300
ibu                                                            34.6897
num_ratings                                                         43
overall_rating                                                      99
beer_style_rating                                                  100
mean_r

# Calculate Final Ranking Score (out of 100)

In [992]:
# Create a new dataframe specifically for this test (Dogfish Head 90 min)
beer_data_dfh90 = beer_data

In [995]:
# Appending the rating scores to the existing dataframe
beer_data_dfh90['review_sim'] = PREF_SCORES
beer_data_dfh90['style_sim'] = SIMILARITY_SCORES
beer_data_dfh90['weighted_rating'] = RATING_SCORES

In [996]:
# Placeholders for User-input weights between review similarity, style similarity, and overall rating
REVIEW_WEIGHT = 0.40
STYLE_WEIGHT = 0.40
RATING_WEIGHT = 0.20

In [997]:
# Applying the weights to each of the score categories
beer_data_dfh90['review_sim'] = [i * REVIEW_WEIGHT for i in beer_data_dfh90['review_sim']] 
beer_data_dfh90['style_sim'] = [i * STYLE_WEIGHT for i in beer_data_dfh90['style_sim']]
beer_data_dfh90['weighted_rating'] = [i * RATING_WEIGHT for i in beer_data_dfh90['weighted_rating']] 

In [998]:
# Calculate fina score for ranking beers
beer_data_dfh90['final_score'] = beer_data_dfh90['style_sim'] + beer_data_dfh90['review_sim'] + beer_data_dfh90['weighted_rating']

In [999]:
# Create new dataframe to sort the top scores and display
ranking_dfh90 = beer_data_dfh90.sort_values(by = 'final_score', ascending=False)
ranking_dfh90[['id','name','beer_style_norm','abv','ibu','calories','overall_rating','beer_style_rating','review_sim','style_sim','weighted_rating','final_score']].head(25)

,id,name,beer_style_norm,abv,ibu,calories,overall_rating,beer_style_rating,review_sim,style_sim,weighted_rating,final_score
1783,1834,Flossmoor Station Killer Kowalski Baltic Porter,Porter,8.00,48.854839,240.0,98,94,36.845834,7.597268,19.667402,64.110504
2223,2283,Southampton Abbot 12,Strong Ale,10.00,36.430233,300.0,99,95,38.673899,1.302846,19.559658,59.536404
2463,2569,OEC Aeris,Brown/Amber Ale,6.20,57.000000,186.0,97,82,36.157861,5.256414,17.647247,59.061522
3661,3963,Stone Sublimely Self Righteous Black IPA,IPA,8.70,90.000000,261.0,100,100,0.000000,37.777163,19.965549,57.742711
2231,2294,Struise XXX Rye Quad Reserva (Bourbon Barrel A...,Strong Ale,10.00,36.430233,300.0,98,94,36.880970,1.203374,19.457949,57.542293
2258,2317,Le Trou du Diable Buteuse Brassin Sp??cial (Br...,Strong Ale,10.00,36.430233,300.0,99,100,36.331456,1.469205,19.482734,57.283394
2252,2312,Mikkeller Nelson Sauvignon,Strong Ale,9.00,36.430233,270.0,100,100,35.471244,2.062699,19.667086,57.201029
2211,2288,Chimay Grande R??serve Vieillie En Barriques (...,Strong Ale,10.50,36.430233,315.0,99,95,36.739521,0.576670,19.368792,56.684983
2219,2279,Lost Abbey Judgment Day,Strong Ale,10.50,36.430233,315.0,99,96,36.145332,0.652784,19.650972,56.449088
3692,3990,Nickel Brook Malevolent Black Imperial India P...,IPA,9.50,90.000000,285.0,99,99,0.000000,36.631640,19.760023,56.391663


In [1000]:
ranking_dfh90.to_csv('./rank_tables/ranking_dfh90.csv')

# Brewery Mapping (TBU)

In [279]:
# Filter the dataframe based on minimum number of breweries and search radius
# Minimum breweries can be either 1, 3, 5 or 10 (although could also allow any value)
MIN_BREWERIES = 3
SEARCH_RADIUS = 5

In [288]:
# Filtered dataset
city_data_filtered = city_data.loc[city_data['breweries_'+str(SEARCH_RADIUS)] >= MIN_BREWERIES]
city_data_filtered = city_data_filtered.sort_values(by = 'brew_per_100k_'+str(SEARCH_RADIUS), ascending=False)[0:25]

In [318]:
city_data_filtered['Rank'] = [i for i in range(1,26)]
city_data_filtered.head(1)

,Rank,city,state,latitude,longitude,July_2016,breweries_5,breweries_10,breweries_25,brew_per_100k_5,...,brew_per_100k_25,ratings_5,ratings_10,ratings_25,review_count_5,review_count_10,review_count_25,price_avg_5,price_avg_10,price_avg_25
746,1,Commerce City,Colorado,39.80832,-104.933868,54869,25,79,187,45.563068,...,340.811752,0,0,0,0,0,0,0,0,0


In [341]:
# Creating the map object to hold the different layers.

# Starting coordinates to load map view.
start_coordinates = (39.965299, -98.266951)

# Create Map object.
map = folium.Map(location=start_coordinates,
                 zoom_start=4,
                tiles = 'Cartodb Positron',
                control_scale = 'True')

# Create layer group for breweries
brew_group = folium.FeatureGroup(name='Breweries per 100k').add_to(map)

import branca

# Add circle and popup info for each city
for row in city_data_filtered.iterrows():
    # Only plot point if lat/long is available.
    if (not np.isnan(row[1]['latitude']) and not np.isnan(row[1]['longitude'])):
        brewery_metadata = """
                <ul>
                    <li><strong>Ranking</strong>: {0}</li>
                    <li><strong>City</strong>: {1}, {2}</li>
                    <li><strong>Population</strong>: {3}</li>
                    <li><strong>Breweries within {4} miles</strong>: {5}</li>
                    <li><strong>Breweries per 100,000 people</strong>: {6}</li>
                </ul>""".format(
            str(row[1]['Rank']), str(row[1]['city']), str(row[1]['state']), str(row[1]['July_2016']), str(SEARCH_RADIUS),
            str(row[1]['breweries_'+str(SEARCH_RADIUS)]), str(row[1]['brew_per_100k_'+str(SEARCH_RADIUS)]))
        iframe = branca.element.IFrame(html=brewery_metadata, width=300, height=150)
        popup = folium.Popup(iframe, max_width=2650)
        folium.CircleMarker(
            location = [row[1]['latitude'], row[1]['longitude']],
            color = 'black', fill_color = 'red',
            radius = 25*row[1]['brew_per_100k_'+str(SEARCH_RADIUS)]/np.max(city_data_filtered['brew_per_100k_'+str(SEARCH_RADIUS)]),
            popup=popup).add_to(brew_group)

In [342]:
folium.LayerControl().add_to(map)
map

In [343]:
# Save Map html
map.save('./brew_count_map.html')

# Heatmap

In [23]:
# Starting heatmap coordinates.
start_coordinates_HM = start_coordinates
radius = 15
blur = 15
min_opacity = 0.1
max_zoom = 13
max_val = 0.8


# Heatmap
map2 = folium.Map(location=start_coordinates_HM,
                 zoom_start=4,
                tiles = 'Cartodb Positron',
                control_scale = 'True')

data = []

for i in range(0,2226):
    lat = brewery_data.iloc[i]['lat']
    lng = brewery_data.iloc[i]['long']
    rtg = brewery_data.iloc[i]['mean_rating']
    
    row = [lat,lng,rtg]
    data.append(row)
    
# Can adjust radius and max_val to change the heatmap concentrations
HeatMap(data = data, name='Beer Rating', radius=radius, blur=blur,
        min_opacity=min_opacity, max_zoom=max_zoom, max_val=max_val).add_to(map2)

folium.LayerControl().add_to(map2)
map2

In [24]:
data

[[45.459109000000005, -123.80389, nan],
 [26.182160999999997, -80.133409999999998, 4.1500000000000004],
 [39.284607000000001, -76.556960000000004, nan],
 [42.108090000000004, -86.418009999999995, nan],
 [42.347974000000001, -71.044629999999998, nan],
 [45.459109000000005, -123.80389, nan],
 [42.108090000000004, -86.418009999999995, nan],
 [45.459109000000005, -123.80389, nan],
 [45.459109000000005, -123.80389, nan],
 [42.816359999999996, -89.640749999999997, nan],
 [41.551456999999999, -87.501429999999999, nan],
 [39.731285999999997, -104.98306000000001, 4.04],
 [40.645099000000002, -73.945031999999998, nan],
 [37.580345999999999, -77.488309999999998, 3.9700000000000002],
 [42.816359999999996, -89.640749999999997, nan],
 [39.284607000000001, -76.556960000000004, nan],
 [33.348593000000001, -111.80833, 4.0800000000000001],
 [38.927343999999998, -105.18746000000002, 3.9199999999999999],
 [45.459109000000005, -123.80389, 3.9399999999999999],
 [41.428162999999998, -73.127290000000002, 3.97

In [9]:
# Playing around with legend.
colormap = cm.LinearColormap(['blue', 'cyan', 'lime', 'yellow', 'red'],
                           vmin=0, vmax=10)
colormap.caption = 'Bucketed Score (Heatmaps)'
colormap